# MA 705 Data Science – Project

**Topic:** Adolescent Fertility Rate (World Bank Indicator)

**Course:** MA 705 Data Science  

**Instructor:** Jiaying Weng

**Student:** Zhongyi Li, Isaiah Gomes

**Date:** 11/15/2025

### Documenting Data Sources

**a. Where the data is coming from and which institution provides it**

The data for this project comes from the World Bank’s World Development Indicators (WDI) database. Specifically, I use the indicator “Adolescent fertility rate (births per 1,000 women ages 15–19)” with the code `SP.ADO.TFRT`. The dataset is provided by the World Bank, an international financial institution that collects official statistics from national statistical offices and other international organizations and then publishes them in a standardized form on its open data website.

**b. Format of the raw data**

The raw data are provided as comma-separated values CSV files. The main file, `API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`, contains a wide table where each row represents a country or aggregate region and the columns include country identifiers followed by one column for each calendar year (for example 1960, 1961, …, 2023, 2024). A second CSV file, `Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`, contains country-level metadata such as region and income group. Both files are plain text CSVs that can be read directly into Python using `pandas`.

**c. Nature of the data (what it contains, how often it is updated, etc.)**

The dataset is a panel (longitudinal) dataset that records adolescent fertility rates by country and year. For each country–year combination, the main variable is the number of births per 1,000 women aged 15–19. The time coverage starts in 1960 and extends to the most recent year available in the current download, with some missing values for certain countries or years. The geographic coverage includes most countries in the world, as well as several regional aggregates defined by the World Bank. The World Development Indicators are generally updated once per year when new national data become available, so the dataset is based on official, regularly updated statistics that are suitable for cross-country and over-time comparisons.


## Dataset Description

In this project, I use data on adolescent fertility rates from the World Bank’s World Development Indicators (WDI). The specific indicator is called “Adolescent fertility rate (births per 1,000 women ages 15–19)” and its code is `SP.ADO.TFRT`. I downloaded the data from the World Bank Open Data website, which is an official public data source provided by the World Bank Group. The World Bank collects and compiles these indicators from national statistical offices and international agencies, and then publishes them in a consistent format for researchers and policymakers.

The main raw data file is named `API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`. This file is a comma-separated values (CSV) file. After skipping the descriptive header lines at the top, each row in the table represents one country or regional aggregate (for example a specific country such as Afghanistan, or a broader region such as “Africa Eastern and Southern”). The first columns identify the country and the indicator through fields such as “Country Name”, “Country Code”, “Indicator Name” and “Indicator Code”. After these identifier columns, each remaining column corresponds to a calendar year, starting from 1960 and continuing up to the most recent years (for example 2023, with an extra mostly empty column for 2024 in this version of the file). The cell values in these year columns are the adolescent fertility rates for that country and year, measured as the number of births per 1,000 women ages 15–19. Some entries are missing, especially for earlier time periods or the newest year.

I also use a second CSV file with additional metadata for each country, called `Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`. In this metadata file, each row again represents a country or region identified by a “Country Code”, and there are extra variables that describe the country. Important examples include the country’s “Region” (such as “Sub-Saharan Africa” or “Latin America & Caribbean”) and its “IncomeGroup” (such as “Low income”, “Lower middle income” or “High income”). There are also some text fields like “SpecialNotes” and “TableName”. Some columns in this metadata file are empty and will be dropped during cleaning. I will merge this metadata with the main indicator file using the country code, so that each observation has both the adolescent fertility rate and the region and income group information attached.

Overall, the nature of the data is a panel (or longitudinal) dataset where the basic unit of observation is a country–year pair. For each country and each year, the main variable of interest is the adolescent fertility rate. The time coverage begins in 1960 and extends through the most recent year available in the current download, which allows me to study long-term trends over several decades. The geographic coverage includes most countries in the world, along with some regional aggregates defined by the World Bank. The World Development Indicators are typically updated once per year when new national statistics become available, so this dataset reflects official and relatively up-to-date information. Because the World Bank harmonizes the data across countries, the dataset is suitable for cross-country comparison and for analyzing how adolescent fertility evolves over time across different regions and income groups.

In the analysis, I will first reshape the main indicator file from wide format (with one column per year) into a tidy long format where each row contains a country code, a year and the corresponding adolescent fertility rate. Then I will join this tidy dataset to the country metadata file so that I can compare adolescent fertility patterns by region and by income group, and use these variables later in visualization and modeling.


### Retrieval of the Raw Data

**a. How the data were retrieved and what was automated**

To retrieve the adolescent fertility data, I first went to the World Bank Open Data website and searched for the indicator with code `SP.ADO.TFRT` (“Adolescent fertility rate (births per 1,000 women ages 15–19)”). On the indicator page, I used the CSV download option and saved the files `API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv` and `Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv` into a local `data/` folder inside my project directory. This initial download step was done manually through the browser and is not automated.

After the files were saved locally, all the later steps were automated in the notebook using Python code. In particular, the notebook automatically locates the `data/` folder and reads the CSV files into `pandas` DataFrames each time the notebook is run. 

In [ ]:
from pathlib import Path
import pandas as pd

# Set up the project and data paths (assuming this notebook is in the "Project 1" folder)
project_dir = Path(".")  # current folder (Project 1)
data_dir = project_dir / "Adolescent fertility rate"

# Paths to the raw CSV files
main_path = data_dir / "API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"
meta_path = data_dir / "Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"

# Read the main indicator file, skipping the descriptive header lines
df_raw = pd.read_csv(main_path, skiprows=4)

# Read the country metadata file
df_meta_raw = pd.read_csv(meta_path)

# Quick checks to confirm that the files loaded correctly
print(df_raw.shape)
print(df_meta_raw.shape)

(266, 70)
(265, 6)


### From Raw Data to Tidy Tabular Data in pandas

**a. Steps to transform the raw data into pandas DataFrames**

The raw adolescent fertility data from the World Bank are initially stored as CSV files in a local `data/` folder. To work with these data in Python, I first loaded the CSV files into `pandas` DataFrames. For the main indicator file (`API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`), I needed to skip four descriptive header lines at the top of the file so that the fifth line would be read as the actual column header row. I used `pd.read_csv(..., skiprows=4)` to achieve this.

After reading the main file, I dropped an extra column called `Unnamed: 69`, which is completely empty and does not contain any meaningful information. The remaining columns include the country identifiers (`Country Name`, `Country Code`, `Indicator Name`, `Indicator Code`) followed by one column for each year from 1960 to 2024. Because this structure is in “wide” format (with one column per year), I reshaped the DataFrame into a tidy “long” format using `pd.melt`. In the long format, each row represents a single country–year pair, with one column holding the year and another column holding the adolescent fertility rate. I renamed this value column to something descriptive such as `Adolescent_Fertility`, converted the `Year` column to an integer type, and removed rows where the fertility rate is missing.

For the metadata file (`Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`), I read it directly into a second DataFrame with `pd.read_csv`. I then kept only the useful columns (`Country Code`, `Region`, `IncomeGroup`, `TableName`, `SpecialNotes`) and dropped the empty `Unnamed: 5` column. Finally, I merged the tidy fertility DataFrame with the metadata DataFrame on `Country Code`, so that each observation includes both the adolescent fertility rate and the region and income group information. At the end of these steps, I obtained a tidy pandas DataFrame with one row per country–year, and columns for the country name, country code, year, adolescent fertility rate, region, and income group.


In [2]:
from pathlib import Path
import pandas as pd

# Base folder = the Project 1 folder where the notebook lives
project_dir = Path.cwd()

# The CSV files are inside the "Adolescent fertility rate" subfolder
data_dir = project_dir / "Adolescent fertility rate"

main_path = data_dir / "API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"
meta_path = data_dir / "Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"

# 1. Read main indicator file (skip descriptive header lines)
df_raw = pd.read_csv(main_path, skiprows=4)

# Drop the empty column if it exists
if "Unnamed: 69" in df_raw.columns:
    df_raw = df_raw.drop(columns=["Unnamed: 69"])

# 2. Convert from wide (one column per year) to long tidy format
id_vars = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
df_long = df_raw.melt(
    id_vars=id_vars,
    var_name="Year",
    value_name="Adolescent_Fertility"
)

# Clean types and drop missing values
df_long["Year"] = df_long["Year"].astype(int)
df_long = df_long.dropna(subset=["Adolescent_Fertility"])

# 3. Read and clean metadata
df_meta = pd.read_csv(meta_path)
df_meta = df_meta[["Country Code", "Region", "IncomeGroup", "TableName", "SpecialNotes"]]

# 4. Merge tidy fertility data with country metadata
df_tidy = df_long.merge(df_meta, on="Country Code", how="left")

df_tidy.head()


,Country Name,Country Code,Indicator Name,Indicator Code,Year,Adolescent_Fertility,Region,IncomeGroup,TableName,SpecialNotes
0,Aruba,ABW,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,44.732000,Latin America & Caribbean,High income,Aruba,NaN
1,Africa Eastern and Southern,AFE,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,135.792572,NaN,NaN,Africa Eastern and Southern,"26 countries, stretching from the Red Sea in t..."
2,Afghanistan,AFG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,140.598000,Middle East & North Africa,Low income,Afghanistan,The reporting period for national accounts dat...
3,Africa Western and Central,AFW,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,161.129238,NaN,NaN,Africa Western and Central,"22 countries, stretching from the westernmost ..."
4,Angola,AGO,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,92.548000,Sub-Saharan Africa,Lower middle income,Angola,The World Bank systematically assesses the app...


### Data Enrichment

**a. Work done to enrich the data (joins, groupby, aggregation)**

After transforming the raw adolescent fertility data into a tidy format, I enriched the dataset by adding country-level information from the metadata file. The tidy fertility DataFrame (`df_long`) contains one row per country–year, with columns for `Country Name`, `Country Code`, `Year`, and `Adolescent_Fertility`. Separately, the metadata file (`Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv`) contains additional variables for each country code, including its World Bank region and income group.

To enrich the fertility data, I first read the metadata CSV into a pandas DataFrame and selected only the relevant columns, such as `Country Code`, `Region`, and `IncomeGroup`. I then performed a left join (merge) on `Country Code`, so that each row in the fertility dataset gained the corresponding region and income group values. This step created an enriched DataFrame (`df_tidy`) that combines the original fertility rates with the contextual information needed for analysis by region and income level.

Once the merge was complete, I used `groupby` and aggregation operations to compute summary statistics. For example, I grouped the enriched data by `Region` and `Year` and calculated the mean adolescent fertility rate within each region–year combination. I also grouped by `IncomeGroup` and `Year` to examine how average adolescent fertility differs across income levels over time. These grouping and aggregation steps produce higher-level views of the data that are easier to interpret than raw country-level values and help highlight broader patterns and trends.

In [5]:
import pandas as pd
from pathlib import Path

# Base folder = Project 1 (where the notebook lives)
project_dir = Path(".")
data_dir = project_dir / "Adolescent fertility rate"

main_path = data_dir / "API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"
meta_path = data_dir / "Metadata_Country_API_SP.ADO.TFRT_DS2_en_csv_v2_14702.csv"

# 1. Read and tidy the main indicator data (assumes df_long already created from df_raw.melt)
df_raw = pd.read_csv(main_path, skiprows=4)
if "Unnamed: 69" in df_raw.columns:
    df_raw = df_raw.drop(columns=["Unnamed: 69"])

id_vars = ["Country Name", "Country Code", "Indicator Name", "Indicator Code"]
df_long = df_raw.melt(
    id_vars=id_vars,
    var_name="Year",
    value_name="Adolescent_Fertility"
)

df_long["Year"] = df_long["Year"].astype(int)
df_long = df_long.dropna(subset=["Adolescent_Fertility"])

# 2. Read metadata and select enrichment variables
df_meta = pd.read_csv(meta_path)
df_meta = df_meta[["Country Code", "Region", "IncomeGroup"]]

# 3. Enrich fertility data by merging with metadata
df_tidy = df_long.merge(df_meta, on="Country Code", how="left")

# 4. Example aggregations: mean fertility by region and year, and by income group and year
region_year_summary = (
    df_tidy
    .groupby(["Region", "Year"], as_index=False)["Adolescent_Fertility"]
    .mean()
    .rename(columns={"Adolescent_Fertility": "Mean_Adolescent_Fertility"})
)

income_year_summary = (
    df_tidy
    .groupby(["IncomeGroup", "Year"], as_index=False)["Adolescent_Fertility"]
    .mean()
    .rename(columns={"Adolescent_Fertility": "Mean_Adolescent_Fertility"})
)

region_year_summary.head(), income_year_summary.head()

(                Region  Year  Mean_Adolescent_Fertility
 0  East Asia & Pacific  1960                  72.397946
 1  East Asia & Pacific  1961                  70.705676
 2  East Asia & Pacific  1962                  70.925324
 3  East Asia & Pacific  1963                  69.998568
 4  East Asia & Pacific  1964                  67.845784,
    IncomeGroup  Year  Mean_Adolescent_Fertility
 0  High income  1960                  65.256616
 1  High income  1961                  64.624326
 2  High income  1962                  63.771337
 3  High income  1963                  63.185477
 4  High income  1964                  62.875419)

### 5. Data Cleaning

In this section I describe the main steps I used to clean and standardize the adolescent fertility dataset in pandas. The goal of the cleaning process was to obtain a tidy DataFrame with clear column names, consistent data types, and a transparent strategy for handling missing values and duplicates.

**i. Renaming columns with explicit column names**

After merging the fertility data with the country metadata, I had a DataFrame called `df_tidy` that still used the original World Bank column names such as `Country Name`, `Country Code`, `Adolescent_Fertility`, `Region`, and `IncomeGroup`. To make the column names easier to work with in Python, I created a cleaned version of this DataFrame with more explicit and consistent names in snake_case. For example, I renamed `Country Name` to `country_name`, `Country Code` to `country_code`, and `Adolescent_Fertility` to `adolescent_fertility`. This renaming step makes the code more readable and reduces the risk of typos.


In [6]:
# Start from the enriched tidy DataFrame
df_clean = df_tidy.rename(
    columns={
        "Country Name": "country_name",
        "Country Code": "country_code",
        "Indicator Name": "indicator_name",
        "Indicator Code": "indicator_code",
        "Adolescent_Fertility": "adolescent_fertility",
        "Region": "region",
        "IncomeGroup": "income_group"
    }
)

In [7]:
# Ensure Year is integer
df_clean["Year"] = df_clean["Year"].astype(int)

# Ensure adolescent_fertility is numeric
df_clean["adolescent_fertility"] = pd.to_numeric(
    df_clean["adolescent_fertility"], errors="coerce"
)

# (Optional) create a decade variable for some summary plots
df_clean["decade"] = (df_clean["Year"] // 10) * 10


In [8]:
# Drop rows where the main outcome is missing
df_clean = df_clean.dropna(subset=["adolescent_fertility"])

# (Optional) fill missing region or income_group with a label
df_clean["region"] = df_clean["region"].fillna("Unknown")
df_clean["income_group"] = df_clean["income_group"].fillna("Unknown")


In [9]:
# Check for duplicate country–year combinations
dupes = df_clean[df_clean.duplicated(subset=["country_code", "Year"], keep=False)]

print("Number of duplicate country-year rows:", len(dupes))

# If duplicates existed, we could drop them as follows:
df_clean = df_clean.drop_duplicates(subset=["country_code", "Year"])


Number of duplicate country-year rows: 0


### Tests for Data Quality

To make sure that my data-cleaning and enrichment steps were performed correctly, I included several simple data-quality tests in the notebook. These tests are implemented using Python `assert` statements and basic summary checks in pandas. They help verify that the final cleaned DataFrame has the structure and content that I expect.

**1. Checking for missing values in key variables**

After cleaning the data, I expect the main outcome variable `adolescent_fertility` to have no missing values, because I explicitly dropped rows where this variable was `NaN`. I also expect that `country_code` and `Year` should always be present, since they identify each observation. I used `assert` statements to verify these conditions. If any of these checks fail, Python will raise an error, which tells me that something went wrong in the earlier cleaning steps.


In [10]:
# df_clean is the final cleaned DataFrame

# Check that the main outcome has no missing values
assert df_clean["adolescent_fertility"].isna().sum() == 0, "Missing values in adolescent_fertility"

# Check that the identifiers are not missing
assert df_clean["country_code"].isna().sum() == 0, "Missing country_code values"
assert df_clean["Year"].isna().sum() == 0, "Missing Year values"


In [11]:
n_rows = len(df_clean)
print("Number of rows in df_clean:", n_rows)

# Sanity check: make sure we have at least 10,000 rows but not more than 100,000
assert 10_000 <= n_rows <= 100_000, "Unexpected number of rows in df_clean"


Number of rows in df_clean: 16960


In [12]:
min_val = df_clean["adolescent_fertility"].min()
max_val = df_clean["adolescent_fertility"].max()
print("Range of adolescent_fertility:", min_val, "to", max_val)

assert min_val >= 0, "Found negative fertility rates"
assert max_val <= 300, "Fertility rates seem unreasonably large"

Range of adolescent_fertility: 0.465 to 229.604


In [13]:
print("Year range:", df_clean["Year"].min(), "to", df_clean["Year"].max())

assert df_clean["Year"].min() >= 1960, "Year values earlier than 1960 found"
assert df_clean["Year"].max() <= 2024, "Year values later than 2024 found"

Year range: 1960 to 2023


In [14]:
duplicate_mask = df_clean.duplicated(subset=["country_code", "Year"], keep=False)
n_dupes = duplicate_mask.sum()
print("Number of duplicate country-year rows:", n_dupes)

assert n_dupes == 0, "Found duplicate country-year combinations in df_clean"


Number of duplicate country-year rows: 0
